<a href="https://colab.research.google.com/github/eunnjji/MultiCampus-EmpAcademy/blob/main/project2/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6조

- 팀명: SYH : super youth hero
- 주제: 기계 시설물 분야 AI 학습용 데이터를 활용하여 기계 시설물의 고장 여부 예측 모델 개발
- 여수진,성은지,홍승표  
- [참고](https://www.aidatahackathon.com/)

## 절차
1. 문제 분석  
 1.1 iris 데이터셋으로 회귀 연습
2. 데이터분석
3. 데이터전처리
4. 트레인, 검증 set 분리
5. 모델 생성
6. 모델 평가 - k겹 교차 검증이나 등등..
7. 모델 튜닝 - 그리드 탐색이나 랜덤 탐색, 앙상블 등
8. 최상의 모델과 오차 분석
9. 테스트 세트로 평가하기
 

In [4]:
# libarary import
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn as skl

# seed setting
np.random.seed(42)
tf.random.set_seed(42)

BASE_DIR = "/content/drive/MyDrive/"
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/daejeon_data'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 형태
csv 파일  
,(콤마)로 분류된 시계열 데이터  
+ current Header  
  - 파일명, 에러 아이디, RPM, 정격전류,
  - 사용전류, Sample Rate 값, R전류의 RMS (RRMS),
  - R 전류의 MEAN (RMEAN), R 전류의 MAX (RMAX),
  - S 전류의 RMS (SRMS), S 전류의 MEAN (SMEAN),
  - S 전류의 MAX (SMAX), T 전류의 RMS (TRMS),
  - T 전류의 MEAN (TMEAN), T 전류의 MAX (TMAX)
+ kimm Header
  - 파일명, Sample Rate 값, Label, Label_No
+ vibration Header
  - 파일명, Sample Rate, Sample Rate, Sample Rate

## 데이터 구조

op-csv : 모터 정상 가동 중 측정한 데이터  
no-csv : 모터가 해당 고장 레이블인 상태에 측정한 데이터

current(전류 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  
kimm(유선진동 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  
vibration(무선진동 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  

|레이블|
|---|
|정상|
|회전체 불평형(unbalance)|
|베어링 불량(bearing)|
|벨트 느슨함(belt)|
|축정렬 불량(misalignment)|
<br>

전류의 경우 \[r,s,t전류 값] target(y)는 \[레이블종류] 특성 \[kw]  
유선진동의 경우 \[] target(y)는 \[레이블종류] 특성 \[kw]  
무선진동의 경우 \[] target(y)는 \[레이블종류] 특성 \[kw]  

target을 원-핫-벡터로


# 1. 문제 분석

지도 학습 문제, 회귀 문제,   
다중 분류 ( 소프트맥스 회귀? ) (예측 결과는 레이블 하나에 해당하도록 나와야 함)  
다층 퍼셉트론 이용(예정)
시계열 데이터 -> RNN ?


# 1.1. Iris Dataset으로 회귀 연습

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
# p197,4장 
type(iris['data']) # numpy.ndarray
iris['data'].shape # (150, 4) 
X = iris['data'][:,3:] 
y = (iris['target']==2).astype(np.int)

In [ ]:
from sklearn.linear_model import LogisticRegression
# 로지스틱회귀, 이진분류
log_reg = LogisticRegression(solver="lbfgs")
log_reg.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
log_reg.predict([[1.7],[1.5]])

array([1, 0])

In [ ]:
#소프트맥스회귀, 다중분류
X = iris["data"][:, (2, 3)]  # petal length, petal width 꽃잎 길이, 너비
y = iris["target"]
softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
softmax_reg.predict([[5, 2]]) # 꽃잎의 길이가 5, 꽃잎 너비가 2cm인 꽃 -> 클래스 2라고 예측

array([2])

In [ ]:
#서포트벡터머신 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64) # feature scailing 필요
svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1,loss='hinge'))
])
svm_clf.fit(X,y)
svm_clf.predict([[5, 2]])

array([1.])

In [ ]:
from sklearn.svm import SVC

from sklearn.linear_model import SGDClassifier
svm_clf2 = Pipeline([
                    ("scaler", StandardScaler()),
                    ("SVC", SVC(kernel='hinge',C=1))
])

In [ ]:
svm_clf3 = Pipeline([
                    ("scaler", StandardScaler()),
                    # ("sgd_clf", SGDClassifier(loss='hinge',alpha= )
])

In [ ]:
#결정트리


In [ ]:
#랜덤포레스트

In [ ]:
#투표기반 분류

In [ ]:
#앙상블

# 2. 데이터 분석


In [ ]:
# !unzip -uq "/content/drive/MyDrive/경진대회_data.zip" -d "압축 푼 파일 저장할 경로"

,로 구분된 csv 파일 형태로 데이터 가져오기

In [ ]:
%cd '/content/drive/MyDrive'
os.getcwd()

'/content/drive/My Drive'

In [5]:
classSet = ("bearing", "belt", "misalignment", "unbalance")

def DatadirLoad(dirname):
    csv_op_dict = {classSet[0]:[], classSet[1]:[], classSet[2]:[],classSet[3]:[]}
    csv_no_dict = {classSet[0]:[], classSet[1]:[], classSet[2]:[],classSet[3]:[]}

    dirpath = os.path.join(DATA_DIR,dirname)
    if os.path.isdir(dirpath):
        for d in os.listdir(dirpath):
            d_path = dirpath+'/'+d
            if os.path.isdir(d_path):
                Error_Class_Lable,kw_Name,*_ = d.split('_')
                
                data_dir_list = [ x for x in os.listdir(d_path) if len(x)<2]
                data_dir_list.sort()
                
                if len(data_dir_list) > 0:
                    for d2 in data_dir_list:
                        d2_path = os.path.join(d_path,d2)

                        # op-csv file # 정상 작동
                        d2_op_path = os.path.join(d2_path,'4.op-csv')
                        d2_op_data = [x for x in os.listdir(d2_op_path) if x.endswith('.csv')]
                        op_csv_path = os.path.join(d2_op_path,d2_op_data[0])
                        csv_op_dict[Error_Class_Lable].append(op_csv_path)
                        
                        # no-csv file # 고장 발생
                        d2_no_path = os.path.join(d2_path,'5.no-csv')
                        d2_no_data = [x for x in os.listdir(d2_no_path) if x.endswith('.csv')]
                        no_csv_path = os.path.join(d2_no_path,d2_no_data[0])
                        csv_no_dict[Error_Class_Lable].append(no_csv_path)
                

    return csv_op_dict, csv_no_dict

In [6]:
def read_csv_dir(path, max_row_size):
    t_pd = None
    update_row_size = max_row_size
    if os.path.isfile(path):
        
        data_Name = os.path.basename(path)
        dir_name,Ekw,dir_num,opno  = os.path.dirname(path).split('/')[6:]
        Error_Class_Lable,kw_Name,*_ = Ekw.split('_')      
        opno = 0 if opno[0] == '4' else 1
        # 정상 : False(0), 고장 : True(1)

        if dir_name[0] == 'c': #current(전류)
            # print('read')
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(15))
            t_pd.drop(3, axis=1, inplace=True)
            
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = Error_Class_Lable if opno == 1 else 'normal'

        elif dir_name[0] == 'k': # kimm(유선진동)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(4))
            t_pd.drop(1, axis=1, inplace=True)
                        
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = Error_Class_Lable if opno else 'normal'

        elif dir_name[0] == 'v': #vibration(무선진동)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(4))
            t_pd.drop(1, axis=1, inplace=True)
            
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = Error_Class_Lable if opno else 'normal'
            
            if (t_pd.values.shape[0]) > max_row_size:
                update_row_size = t_pd.values.shape[0]
            elif (t_pd.values.shape[0]) < max_row_size:
                update_row_size = max_row_size
                tmp = np.zeros((int(update_row_size - t_pd.values.shape[0]),int(t_pd.values.shape[1])))
                new_t = np.vstack([t_pd.values,tmp])
                return new_t.tolist(), update_row_size
        else: 
            print('error')
            print('dir_category : {}, lable : {}, op: {}'.format(dir_name, Error_Class_Lable,opno))
            return 0
    return t_pd.values.tolist(), update_row_size
    

In [9]:
# dataset 생성
def makeDataset(op_dict, no_dict, shape_op = 0):
    """
    input: 정상data의 path list, 오류 data의 path list
    output: numpy의 ndarray 타입 형태로 dataset 반환
    shape_op : 0이면 (38200, 60, 5) 형태, 1이면 (38200(data개수 * row/60한 값), 5(특성 수), 60)
    """
    tmp = []
    max_row_size = 0
    for k,v in op_dict.items():
        print(k)
        for p in v:
            data, max = read_csv_dir(p, max_row_size)
            tmp.append(data)
            max_row_size = max
    for k,v in no_dict.items():
        print(k)
        for p in v:
            data, max = read_csv_dir(p, max_row_size)
            tmp.append(data)
            max_row_size = max
    if shape_op == 0:
        dataset = np.stack(tmp)
        dataset.reshape((-1,60,5))
    else:
        dataset = np.stack(tmp,axis=2)
        dataset.reshape((-1,5,60))
    
    return dataset  

In [ ]:
current_op_csv,current_no_csv = DatadirLoad("current(전류)")
kimm_op_csv, kimm_no_csv = DatadirLoad("kimm(유선진동)")
vibration_op_csv, vibration_no_csv = DatadirLoad("vibration(무선진동)")

In [ ]:
# reac_csv Test 코드 (정상가동데이터)
# 경로에 맞춰서 csv 파일 하나만 읽어옴
tp = '/content/drive/MyDrive/Colab Notebooks/daejeon_data/current(전류)/bearing_11kw_1/1/4.op-csv/STFCB-20201012-0105-0123_20210106_090604_002.csv'
t_pd = pd.read_csv(tp, header=None,encoding='utf-8-sig',skiprows=range(15))
t_pd

,0,1,2,3
0,11.531250,14.293945,25.304688,NaN
1,7.367188,18.097656,25.224609,NaN
2,3.443359,21.260742,24.624023,NaN
3,-1.721680,22.742188,21.340820,NaN
4,-5.765625,24.704102,19.539062,NaN
...,...,...,...,...
5995,21.500977,1.521484,22.021484,NaN
5996,18.658203,5.605469,23.422852,NaN
5997,15.094727,10.770508,25.144531,NaN
5998,9.929688,16.135742,25.625000,NaN


In [ ]:
# reac_csv Test 코드 (오류데이터)
tp = '/content/drive/MyDrive/project/daejeon_data/current(전류)/bearing_11kw_1/1/5.no-csv/STFCB-20201012-0105-0123_20210106_093704_002.csv'
read_csv_dir(tp)

In [ ]:
current_dataset = makeDataset(current_op_csv,current_no_csv)
current_dataset.shape

In [ ]:
#확인용, 192 이후 데이터(고장데이터) 레이블(5번째 특성) 값 확인 코드
pd.DataFrame(current_dataset[:,5,192:]) 

In [ ]:
kimm_dataset = makeDataset(kimm_op_csv,kimm_no_csv)
kimm_dataset.shape

In [ ]:
#확인용, 192 이후 데이터(고장데이터) 레이블(3번째 특성) 값 확인 코드
pd.DataFrame(kimm_dataset[:,3,192:]) 

In [ ]:
vibration_dataset = makeDataset(vibration_op_csv, vibration_no_csv)
vibration_dataset.shape

In [ ]:
#확인용, 192 이후 데이터(고장데이터) 레이블(3번째 특성) 값 확인 코드
pd.DataFrame(vibration_dataset[:,3,192:]) 